**Obtaining Files for Analysis**

In [194]:
file_names = [
    "aro_categories.tsv",
    "aro_categories_index.tsv",
    "aro_index.tsv",
    "CARD-Download-README.txt",
    "card.json",
    "nucleotide_fasta_protein_homolog_model.fasta",
    "nucleotide_fasta_protein_knockout_model.fasta",
    "nucleotide_fasta_protein_overexpression_model.fasta",
    "nucleotide_fasta_protein_variant_model.fasta",
    "nucleotide_fasta_rRNA_gene_variant_model.fasta",
    "protein_fasta_protein_homolog_model.fasta",
    "protein_fasta_protein_knockout_model.fasta",
    "protein_fasta_protein_overexpression_model.fasta",
    "protein_fasta_protein_variant_model.fasta",
    "shortname_antibiotics.tsv",
    "shortname_pathogens.tsv",
    "snps.txt",
]

In [195]:
def extract_fasta_files(file_names):
    # Create a dictionary to store FASTA files based on their suffix
    fasta_files_dict = {}
    
    for file in file_names:
        # Extract the suffix of the file name
        suffix = file.split('.')[-1]
        
        # Check if the suffix is 'fasta'
        if suffix == 'fasta':
            # Check if the suffix is already a key in the dictionary
            if suffix in fasta_files_dict:
                fasta_files_dict[suffix].append(file)
            else:
                # If not, create a new key and store the file name in a list
                fasta_files_dict[suffix] = [file]
    
    return fasta_files_dict

result = extract_fasta_files(file_names)
print(result)

{'fasta': ['nucleotide_fasta_protein_homolog_model.fasta', 'nucleotide_fasta_protein_knockout_model.fasta', 'nucleotide_fasta_protein_overexpression_model.fasta', 'nucleotide_fasta_protein_variant_model.fasta', 'nucleotide_fasta_rRNA_gene_variant_model.fasta', 'protein_fasta_protein_homolog_model.fasta', 'protein_fasta_protein_knockout_model.fasta', 'protein_fasta_protein_overexpression_model.fasta', 'protein_fasta_protein_variant_model.fasta']}


**Analysis of CARD using Biopython**

In [196]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


As described in Biopython: The combination of homolog fasta files below uses the Bio.SeqIO.parse(...) function, which is used to loop over all the records in a file as SeqRecord objects. TheS eqRecord (SequenceRecord) class is defined in the Bio.SeqRecordmodule. This class allows higher level features such as identifiers and features to be associated with a sequence (see Chapter3), and is the basic data type for the Bio.SeqIO sequence input/output interface (seeChapter5). https://biopython.org/DIST/docs/tutorial/Tutorial.pdf

In [224]:
# Obtaining tools
from Bio import SeqIO
import pandas as pd

# Defining a funciton to convert fasta files into a dataframe
def fasta_to_dataframe(fasta_file):
    records = SeqIO.parse(fasta_file, "fasta")
    data = []
    for record in records:
        parts = record.id.split('|')
        aro_id = parts[-1].split(':')[0]
        other_identifiers = '|'.join(parts[:-1])
        data.append((aro_id, other_identifiers, str(record.seq)))
    df = pd.DataFrame(data, columns=['ARO', 'Other_identifiers', 'Sequence'])
    return df

# Defining a function to merge converted fasta dataframe
def merge_fasta_files(file1, file2):
    df1 = fasta_to_dataframe(file1)
    df2 = fasta_to_dataframe(file2)
    merged_df = pd.merge(df1, df2, on='ARO', how='outer')
    
    # Drop NaN values not needed since dataframe contains homolog (complete) data --> useful for other forms of Fasta files. 
   # merged_df.dropna(inplace=True)
    return merged_df

# File paths for homolog Fasta files
file1 = "nucleotide_fasta_protein_homolog_model.fasta"
file2 = "protein_fasta_protein_homolog_model.fasta"

# Merge the files
merged_df = merge_fasta_files(file1, file2)

In [225]:
# Organize dataframe and rename the columns
merged_df.rename(columns={'Other_identifiers_x': 'Other_identifiers_nucleotide',
                          'Other_identifiers_y': 'Other_identifiers_protein',
                          'Sequence_x': 'Sequence_nucleotide',
                          'Sequence_y': 'Sequence_protein'}, inplace=True)

# Renaming the ARO term to match CARD literature, see the 
dict = {'ARO':'CARD_Short_Name'}
merged_df.rename(columns=dict,
                 inplace=True)

In [226]:
merged_df.head(5)

,CARD_Short_Name,Other_identifiers_nucleotide,Sequence_nucleotide,Other_identifiers_protein,Sequence_protein
0,CblA-1,gb|GQ343019.1|+|132-1023|ARO:3002999,ATGAAAGCATATTTCATCGCCATACTTACCTTATTCACTTGTATAG...,gb|ACT97415.1|ARO:3002999,MKAYFIAILTLFTCIATVVRAQQMSELENRIDSLLNGKKATVGIAV...
1,SHV-52,gb|HQ845196.1|+|0-861|ARO:3001109,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCGCCCTGC...,gb|AEJ08681.1|ARO:3001109,MRYIRLCIISLLAALPLAVHASPQPLEQIKQSESQLSGRVGMIEMD...
2,dfrF,gb|AF028812.1|+|392-887|ARO:3002867,ATGATAGGTTTGATTGTTGCGAGGTCAAAGAATAATGTTATAGGCA...,gb|AAD01868.1|ARO:3002867,MIGLIVARSKNNVIGKNGNIPWKIKGEQKQFRELTTGNVVIMGRKS...
3,CTX-M-130,gb|JX017365.1|+|244-1120|ARO:3001989,ATGGTGACAAAGAGAGTGCAACGGATGATGTTCGCGGCGGCGGCGT...,gb|AFJ59957.1|ARO:3001989,MVTKRVQRMMFAAAACIPLLLGSAPLYAQTSAVQQKLAALEKSSGG...
4,NDM-6,gb|JN967644.1|+|0-813|ARO:3002356,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,gb|AEX08599.1|ARO:3002356,MELPNIMHPVAKLSTALAAALMLSGCMPGEIRPTIGQQMETGDQRF...


In [228]:
# Using Python documentation to develop pattern used to extract specific identifiers for proteins and nucleotides: https://docs.python.org/3/library/re.html and https://www.regular-expressions.info/

# Extraction pattern for nucleotide identifiers
pattern_nucleotide = r'(\w+)\|(\w+)\.(\d+)\|([+-])\|(\d+-\d+)'

# Extraction pattern for protein identifiers
pattern_protein = r'(\w+)\|(\w+)\.(\d+)\|ARO:(\d+)'

# Extracting information from the "Other_identifiers_nucleotide" column
merged_df[['Database_nucleotide', 'Accession_Number_Nucleotide', 'Sequence_Version_Nucleotide', 'Strand_Orientation_Nucleotide', 'Sequence_Position_Nucleotide']] = merged_df['Other_identifiers_nucleotide'].str.extract(pattern_nucleotide)

# Extracting information from the "Other_identifiers_protein" column
merged_df[['Database_protein', 'Accession_Number_protein', 'Sequence_Version_protein', 'ARO']] = merged_df['Other_identifiers_protein'].str.extract(pattern_protein)
merged_df.head()

,CARD_Short_Name,Other_identifiers_nucleotide,Sequence_nucleotide,Other_identifiers_protein,Sequence_protein,Database_nucleotide,Accession_Number_Nucleotide,Sequence_Version_Nucleotide,Strand_Orientation_Nucleotide,Sequence_Position_Nucleotide,Database_protein,Accession_Number_protein,Sequence_Version_protein,ARO
0,CblA-1,gb|GQ343019.1|+|132-1023|ARO:3002999,ATGAAAGCATATTTCATCGCCATACTTACCTTATTCACTTGTATAG...,gb|ACT97415.1|ARO:3002999,MKAYFIAILTLFTCIATVVRAQQMSELENRIDSLLNGKKATVGIAV...,gb,GQ343019,1,+,132-1023,gb,ACT97415,1,3002999
1,SHV-52,gb|HQ845196.1|+|0-861|ARO:3001109,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCGCCCTGC...,gb|AEJ08681.1|ARO:3001109,MRYIRLCIISLLAALPLAVHASPQPLEQIKQSESQLSGRVGMIEMD...,gb,HQ845196,1,+,0-861,gb,AEJ08681,1,3001109
2,dfrF,gb|AF028812.1|+|392-887|ARO:3002867,ATGATAGGTTTGATTGTTGCGAGGTCAAAGAATAATGTTATAGGCA...,gb|AAD01868.1|ARO:3002867,MIGLIVARSKNNVIGKNGNIPWKIKGEQKQFRELTTGNVVIMGRKS...,gb,AF028812,1,+,392-887,gb,AAD01868,1,3002867
3,CTX-M-130,gb|JX017365.1|+|244-1120|ARO:3001989,ATGGTGACAAAGAGAGTGCAACGGATGATGTTCGCGGCGGCGGCGT...,gb|AFJ59957.1|ARO:3001989,MVTKRVQRMMFAAAACIPLLLGSAPLYAQTSAVQQKLAALEKSSGG...,gb,JX017365,1,+,244-1120,gb,AFJ59957,1,3001989
4,NDM-6,gb|JN967644.1|+|0-813|ARO:3002356,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,gb|AEX08599.1|ARO:3002356,MELPNIMHPVAKLSTALAAALMLSGCMPGEIRPTIGQQMETGDQRF...,gb,JN967644,1,+,0-813,gb,AEX08599,1,3002356


In [229]:
# Dropping the Other_identifiers column, the data within these columns is already present in the dataframe
merged_df.drop(['Other_identifiers_nucleotide','Other_identifiers_protein'], axis = 1, inplace = True)
merged_df.head()

,CARD_Short_Name,Sequence_nucleotide,Sequence_protein,Database_nucleotide,Accession_Number_Nucleotide,Sequence_Version_Nucleotide,Strand_Orientation_Nucleotide,Sequence_Position_Nucleotide,Database_protein,Accession_Number_protein,Sequence_Version_protein,ARO
0,CblA-1,ATGAAAGCATATTTCATCGCCATACTTACCTTATTCACTTGTATAG...,MKAYFIAILTLFTCIATVVRAQQMSELENRIDSLLNGKKATVGIAV...,gb,GQ343019,1,+,132-1023,gb,ACT97415,1,3002999
1,SHV-52,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCGCCCTGC...,MRYIRLCIISLLAALPLAVHASPQPLEQIKQSESQLSGRVGMIEMD...,gb,HQ845196,1,+,0-861,gb,AEJ08681,1,3001109
2,dfrF,ATGATAGGTTTGATTGTTGCGAGGTCAAAGAATAATGTTATAGGCA...,MIGLIVARSKNNVIGKNGNIPWKIKGEQKQFRELTTGNVVIMGRKS...,gb,AF028812,1,+,392-887,gb,AAD01868,1,3002867
3,CTX-M-130,ATGGTGACAAAGAGAGTGCAACGGATGATGTTCGCGGCGGCGGCGT...,MVTKRVQRMMFAAAACIPLLLGSAPLYAQTSAVQQKLAALEKSSGG...,gb,JX017365,1,+,244-1120,gb,AFJ59957,1,3001989
4,NDM-6,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,MELPNIMHPVAKLSTALAAALMLSGCMPGEIRPTIGQQMETGDQRF...,gb,JN967644,1,+,0-813,gb,AEX08599,1,3002356


**Reviewing Content** using CblA-1 (row 0) as an example

gb: This indicates the type of identifier. In this case, it refers to GenBank, a widely used nucleotide sequence database.

GQ343019.1: This is the accession number, a unique identifier assigned to a sequence record in the GenBank database. It allows users to quickly locate and retrieve specific sequences.

+: This symbol indicates the strand orientation of the sequence. In this case, the plus sign (+) typically denotes the forward or sense strand.

132-1023: This represents the position of the sequence within the genome or the length of the sequence. In this example, it likely indicates that the sequence starts at position 132 and ends at position 1023.

ARO:3002999: This part of the identifier is specific to the Antibiotic Resistance Ontology (ARO). It provides information about antibiotic resistance genes or mechanisms associated with the sequence. 
_______________________________________________________________________________________________________________________________________________________________________________________________________________________
Investigation of the second term in the identifier "gb|GQ343019.1|+|132-1023|ARO:3002999" is "GQ343019.1". This part refers to the accession number of the sequence. Here's what each component of this term typically signifies:

GQ: This part of the accession number usually represents the GenBank division where the sequence is stored. In this case, "GQ" could indicate a specific category or source of the sequence within the GenBank database.

343019: This is a unique numerical identifier assigned to the sequence within its division or category. It's used to distinguish this sequence from others within the same division.

.1: The decimal portion of the accession number often indicates the version of the sequence. When a sequence undergoes updates or revisions, a new version is assigned to it. The version number allows users to track changes to the sequence over time.
_______________________________________________________________________________________________________________________________________________________________________________________________________________________
Information regarding pattern used for extraction of identifiers from columns:

(\w+): Matches one or more word characters (alphanumeric or underscore), capturing the GenBank division.

\|: Matches the pipe character.

(\w+)\.(\d+): Matches the alphanumeric characters followed by a dot and then more digits, capturing the numerical identifier and version.

\|: Matches the pipe character.

([+-]): Matches either a plus or minus sign, capturing the strand information.

\|: Matches the pipe character.

(\d+-\d+): Matches digits followed by a hyphen and then more digits, capturing the position range.

\|ARO:(\d+): Matches "|ARO:" followed by digits, capturing the ARO identifier.

**Investigating the Dataframe**

In [230]:
#determine the type
merged_df.dtypes

CARD_Short_Name                  object
Sequence_nucleotide              object
Sequence_protein                 object
Database_nucleotide              object
Accession_Number_Nucleotide      object
Sequence_Version_Nucleotide      object
Strand_Orientation_Nucleotide    object
Sequence_Position_Nucleotide     object
Database_protein                 object
Accession_Number_protein         object
Sequence_Version_protein         object
ARO                              object
dtype: object

In [231]:
merged_df.describe

<bound method NDFrame.describe of      CARD_Short_Name                                Sequence_nucleotide  \
0             CblA-1  ATGAAAGCATATTTCATCGCCATACTTACCTTATTCACTTGTATAG...   
1             SHV-52  ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCGCCCTGC...   
2               dfrF  ATGATAGGTTTGATTGTTGCGAGGTCAAAGAATAATGTTATAGGCA...   
3          CTX-M-130  ATGGTGACAAAGAGAGTGCAACGGATGATGTTCGCGGCGGCGGCGT...   
4              NDM-6  ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...   
...              ...                                                ...   
4799         TEM-103  ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...   
4800          TEM-61  ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...   
4801        OKP-B-14  TGCCTTATCTCCCTGATTGCCGCCCTGCCACTGGCGGTATTCGCCA...   
4802         OXA-105  ATGAATAAATATTTTACTTGCTATGTGTTTGCCTCTCTTTTTCTTT...   
4803            Mdtq  ATGAAATTGATATTAAATAAAAGCGTGCTGGCGGCATTGCCCCTGG...   

                                       Sequence_protein Database_

In [232]:
merged_df["CARD_Short_Name"].value_counts() 

PDC-358     1
OKP-B-21    1
OXA-565     1
CMY-73      1
CatU        1
           ..
CARB-28     1
mel         1
mdsA        1
IMI-17      1
OXA-561     1
Name: CARD_Short_Name, Length: 4804, dtype: int64

The value counts demonstrates how there is one row deticated to each CARD_Short_Name term

In [233]:
from Bio.Data import CodonTable

# Access the standard/universal codon table
standard_table = CodonTable.unambiguous_dna_by_name["Standard"]

print(standard_table)

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

This table shows the three letter combinations of nucleotides that make up protein sequences. The percentage of amino acids and their orientation within a protein denote some to the proteins characteristics.
**Can be used as a figure to demonstrate conversion of nucleotides to amino acid sequence**

In [240]:
def calculate_lengths(row):
    lengths = {}
    lengths['Sequence_nucleotide'] = len(row['Sequence_nucleotide'])
    lengths['Sequence_protein'] = len(row['Sequence_protein'])
    return lengths

# Iterating through each row
for index, row in merged_df.iterrows():
    lengths = calculate_lengths(row)
    print(f"CARD_Short_Name: {row['CARD_Short_Name']}")
    for key, value in lengths.items():
        print(f"{key}: {value}")
    print()

CARD_Short_Name: CblA-1
Sequence_nucleotide: 891
Sequence_protein: 296

CARD_Short_Name: SHV-52
Sequence_nucleotide: 861
Sequence_protein: 286

CARD_Short_Name: dfrF
Sequence_nucleotide: 495
Sequence_protein: 164

CARD_Short_Name: CTX-M-130
Sequence_nucleotide: 876
Sequence_protein: 291

CARD_Short_Name: NDM-6
Sequence_nucleotide: 813
Sequence_protein: 270

CARD_Short_Name: ACT-35
Sequence_nucleotide: 1146
Sequence_protein: 381

CARD_Short_Name: CARB-5
Sequence_nucleotide: 897
Sequence_protein: 298

CARD_Short_Name: Erm(34)
Sequence_nucleotide: 846
Sequence_protein: 281

CARD_Short_Name: TEM-126
Sequence_nucleotide: 861
Sequence_protein: 286

CARD_Short_Name: LRA-12
Sequence_nucleotide: 882
Sequence_protein: 293

CARD_Short_Name: TEM-72
Sequence_nucleotide: 861
Sequence_protein: 286

CARD_Short_Name: TEM-59
Sequence_nucleotide: 831
Sequence_protein: 277

CARD_Short_Name: KPC-10
Sequence_nucleotide: 882
Sequence_protein: 293

CARD_Short_Name: OXA-212
Sequence_nucleotide: 825
Sequence_pr

The output shows the protein and amino acid sequence length for each row